In [1]:
import pandas as pd
import sqlite3
import sql

sql.style = 'plain'

%load_ext sql
connection=sqlite3.connect("leetcode_database.db")
print("Successfully connected to SQL database")


# Connect to SQLite
%sql sqlite:///leetcode_database.db
print("success")

Successfully connected to SQL database
success


In [2]:
%%sql
-- Drop tables if they already exist
DROP TABLE IF EXISTS Transactions;
DROP TABLE IF EXISTS Visits;

-- Create Visits table
CREATE TABLE Visits (
    visit_id INT PRIMARY KEY,
    customer_id INT
);

-- Insert data into Visits
INSERT INTO Visits (visit_id, customer_id) VALUES
(1, 23),
(2, 9),
(4, 30),
(5, 54),
(6, 96),
(7, 54),
(8, 54);

 * sqlite:///leetcode_database.db
Done.
Done.
Done.
7 rows affected.


[]

In [3]:
%%sql
-- Create Transactions table
CREATE TABLE Transactions (
    transaction_id INT PRIMARY KEY,
    visit_id INT,
    amount INT
);

-- Insert data into Transactions
INSERT INTO Transactions (transaction_id, visit_id, amount) VALUES
(2, 5, 310),
(3, 5, 300),
(9, 5, 200),
(12, 1, 910),
(13, 2, 970);

 * sqlite:///leetcode_database.db
Done.
5 rows affected.


[]

In [4]:
%%sql
SELECT * FROM Visits

 * sqlite:///leetcode_database.db
Done.


visit_id,customer_id
1,23
2,9
4,30
5,54
6,96
7,54
8,54


In [5]:
%%sql
SELECT * FROM Transactions

 * sqlite:///leetcode_database.db
Done.


transaction_id,visit_id,amount
2,5,310
3,5,300
9,5,200
12,1,910
13,2,970


###### PS : Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

* Find out customer_ids of people who made the visit but didn't made any transaction
* How many number of times, they did the same - COUNT function
* Mean, thier visit_id is not present in Transaction Table
* Then group by customer_id to count how many such "non-transaction visits" each user made.
* JOIN - LEFT JOIN ofc, since you need all the customer_id and match it with Transaction table to see which customers have made the transaction (present in the transaction table) or not

In [39]:
%%sql

SELECT *
FROM Visits v
LEFT JOIN Transactions t
ON v.visit_id=t.visit_id

 * sqlite:///leetcode_database.db
Done.


visit_id,customer_id,transaction_id,visit_id_1,amount
1,23,12,1,910
2,9,13,2,970
4,30,None,None,None
5,54,2,5,310
5,54,3,5,300
5,54,9,5,200
6,96,None,None,None
7,54,None,None,None
8,54,None,None,None


* LEFT JOIN

In [43]:
%%sql

SELECT v.customer_id, t.visit_id AS visited_transacted_id  -- visited and made transaction
FROM Visits v
LEFT JOIN Transactions t
ON v.visit_id=t.visit_id

 * sqlite:///leetcode_database.db
Done.


customer_id,visited_transacted_id
23,1
9,2
30,None
54,5
54,5
54,5
96,None
54,None
54,None


* customer no 30, doesn't have a visted_transacted id
* customer no 96, doesn't have a visited_transacted id
* Similarly, for 54 coming twice and not making any transaction

* Filter visit_id not present in Transaction Table using WHERE column IS NULL

In [35]:
%%sql

SELECT v.customer_id, t.visit_id
FROM Visits v
LEFT JOIN Transactions t
ON v.visit_id=t.visit_id
WHERE t.visit_id IS NULL

 * sqlite:///leetcode_database.db
Done.


customer_id,visit_id
30,None
96,None
54,None
54,None


* COUNT of each customer who visited but didn't made any transaction

In [46]:
%%sql

SELECT v.customer_id, t.visit_id, COUNT(*) AS count_of_window_shopping
FROM Visits v
LEFT JOIN Transactions t
ON v.visit_id=t.visit_id
WHERE t.visit_id IS NULL
GROUP BY v.customer_id
ORDER BY count_of_window_shopping DESC

 * sqlite:///leetcode_database.db
Done.


customer_id,visit_id,count_of_window_shopping
54,None,2
96,None,1
30,None,1
